**SINGLE MODEL**

In [1]:


# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report, roc_auc_score






In [2]:
!pip install catboost lightgbm

In [3]:
#For 16 class
# Import necessary libraries
import numpy as np
import pandas as pd
mal_df = pd.read_csv("data.csv")

#For 15 class
#mal_df = pd.read_csv("/content/drive/MyDrive/Research/DataseT/Malware_DataseT.csv")

#mal_df = mal_df.drop_duplicates()
mal_df.shape

mal_df['Class'].value_counts()
mal_df.bfill(inplace=True)

## Converting data types
mal_df["Class"] = mal_df["Class"].astype("category")

y = mal_df["Class"]
X = mal_df.drop(columns=["Category", "Class"])

def category(column):
  return column.split("-")[0] if "-" in column else column

def category_name(column):
  return column.split("-")[1] if "-" in column else column

mal_df["category"] = mal_df["Category"].apply(category)

mal_df["category"].value_counts()

# Creating a column with the names of the variants
mal_df["category_name"] = mal_df["Category"].apply(category_name)
mal_df["category_name"].value_counts()

X.head(5)




,pslist.nproc,pslist.nppid,pslist.avg_threads,pslist.nprocs64bit,pslist.avg_handlers,dlllist.ndlls,dlllist.avg_dlls_per_proc,handles.nhandles,handles.avg_handles_per_proc,handles.nport,...,svcscan.nservices,svcscan.kernel_drivers,svcscan.fs_drivers,svcscan.process_services,svcscan.shared_process_services,svcscan.interactive_process_services,svcscan.nactive,callbacks.ncallbacks,callbacks.nanonymous,callbacks.ngeneric
0,45,17,10.555556,0,202.844444,1694,38.500000,9129,212.302326,0,...,389,221,26,24,116,0,121,87,0,8
1,47,19,11.531915,0,242.234043,2074,44.127660,11385,242.234043,0,...,392,222,26,24,118,0,122,87,0,8
2,40,14,14.725000,0,288.225000,1932,48.300000,11529,288.225000,0,...,395,222,26,27,118,0,120,88,0,8
3,32,13,13.500000,0,264.281250,1445,45.156250,8457,264.281250,0,...,395,222,26,27,118,0,120,88,0,8
4,42,16,11.452381,0,281.333333,2067,49.214286,11816,281.333333,0,...,392,222,26,24,118,0,124,87,0,8


In [4]:
# Define the label encoder
def label_encoder(column):
    lencode = LabelEncoder().fit(column)
    print(column.name, lencode.classes_)
    return lencode.transform(column)


# Encoding the columns
mal_df["category"] = label_encoder(mal_df["category"])
mal_df["category_name"] = label_encoder(mal_df["category_name"])
mal_df["class"] = label_encoder(mal_df["Class"])

# Drop the Category and Class columns
mal_df.drop(["Category", "Class"], axis=1, inplace=True)

category ['Benign' 'Ransomware' 'Spyware' 'Trojan']
category_name ['180solutions' 'Ako' 'Benign' 'CWS' 'Conti' 'Emotet' 'Gator' 'Maze'
 'Pysa' 'Reconyc' 'Refroso' 'Scar' 'Shade' 'TIBS' 'Transponder' 'Zeus']
Class ['Benign' 'Malware']


In [5]:
# Separate features and target
X = mal_df.drop(["category_name", "category", "class"], axis=1)
y = mal_df["category_name"]

# Standardize the data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Initialize all the classifiers for 16-class classification
adaboost = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=20), n_estimators=50)
logreg = LogisticRegression(penalty="l2", C=10, max_iter=10000)
svc = SVC(kernel="linear", random_state=42)
dt = DecisionTreeClassifier(random_state=42, min_samples_leaf=10, min_samples_split=5)
rf = RandomForestClassifier()
xgb = XGBClassifier(use_label_encoder=False, eval_metric="mlogloss",objective="multi:softmax", num_class=len(y.unique()), random_state=42)
#xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
catboost = CatBoostClassifier(verbose=0)
lightgbm = LGBMClassifier()

# Create a list of tuples called classifiers which stores thee classifiers

classifiers = [
    ("Adaboost", adaboost),
     ("Logistic Regression", logreg),
    ("Support Vector", svc),
    ("Decision Tree", dt),
    ("Random Forest", rf),
    ("XGBoost", xgb),
    ("CatBoost", catboost),
    ("LightGBM", lightgbm)
]

metrics_sbtsk3_cols = ["Model Name", "Accuracy Score", "F1 Score", "Precision Score", "Recall Score"]
metrics_sbtsk3 = pd.DataFrame(columns=metrics_sbtsk3_cols)

X.head(5)

,pslist.nproc,pslist.nppid,pslist.avg_threads,pslist.nprocs64bit,pslist.avg_handlers,dlllist.ndlls,dlllist.avg_dlls_per_proc,handles.nhandles,handles.avg_handles_per_proc,handles.nport,...,svcscan.nservices,svcscan.kernel_drivers,svcscan.fs_drivers,svcscan.process_services,svcscan.shared_process_services,svcscan.interactive_process_services,svcscan.nactive,callbacks.ncallbacks,callbacks.nanonymous,callbacks.ngeneric
0,45,17,10.555556,0,202.844444,1694,38.500000,9129,212.302326,0,...,389,221,26,24,116,0,121,87,0,8
1,47,19,11.531915,0,242.234043,2074,44.127660,11385,242.234043,0,...,392,222,26,24,118,0,122,87,0,8
2,40,14,14.725000,0,288.225000,1932,48.300000,11529,288.225000,0,...,395,222,26,27,118,0,120,88,0,8
3,32,13,13.500000,0,264.281250,1445,45.156250,8457,264.281250,0,...,395,222,26,27,118,0,120,88,0,8
4,42,16,11.452381,0,281.333333,2067,49.214286,11816,281.333333,0,...,392,222,26,24,118,0,124,87,0,8


In [6]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

import time

# Add columns for training and inference time
metrics_sbtsk3 = pd.DataFrame(columns=[
    "Model Name", "Accuracy Score", "F1 Score", "Precision Score", "Recall Score",
    "Training Time (s)", "Inference Time (s)"
])

for classifier_name, classifier in classifiers:
    print(f"\nClassifier: {classifier_name}...")

    # Time the training
    start_train = time.time()
    classifier.fit(X_train, y_train)
    end_train = time.time()
    training_time = end_train - start_train

    # Time the inference
    start_infer = time.time()
    y_pred = classifier.predict(X_test)
    end_infer = time.time()
    inference_time = end_infer - start_infer

    # Print classification report
    print(classification_report(y_test, y_pred))

    # Collect metrics with timing
    new_metrics = pd.DataFrame([{
        "Model Name": classifier_name,
        "Accuracy Score": accuracy_score(y_test, y_pred),
        "F1 Score": f1_score(y_test, y_pred, average='weighted'),
        "Precision Score": precision_score(y_test, y_pred, average='weighted'),
        "Recall Score": recall_score(y_test, y_pred, average='weighted'),
        "Training Time (s)": round(training_time, 4),
        "Inference Time (s)": round(inference_time, 4)
    }])

    # Append to the metrics table
    metrics_sbtsk3 = pd.concat([metrics_sbtsk3, new_metrics], ignore_index=True)

# Show final results
metrics_sbtsk3



Classifier: Adaboost...


/home/azwad/anaconda3/envs/Final/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


              precision    recall  f1-score   support

           0       0.46      0.36      0.40       442
           1       0.36      0.33      0.35       389
           2       1.00      1.00      1.00      5790
           3       0.40      0.38      0.39       401
           4       0.44      0.40      0.42       408
           5       0.51      0.53      0.52       375
           6       0.57      0.70      0.63       424
           7       0.57      0.53      0.55       401
           8       0.47      0.43      0.45       349
           9       0.63      0.61      0.62       334
          10       0.76      0.73      0.75       437
          11       0.49      0.62      0.55       393
          12       0.42      0.46      0.44       404
          13       0.83      0.71      0.77       280
          14       0.47      0.49      0.48       483
          15       0.43      0.47      0.45       410

    accuracy                           0.75     11720
   macro avg       0.55   

/tmp/ipykernel_10969/3647427165.py:42: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_sbtsk3 = pd.concat([metrics_sbtsk3, new_metrics], ignore_index=True)


              precision    recall  f1-score   support

           0       0.26      0.09      0.13       442
           1       0.24      0.15      0.19       389
           2       1.00      1.00      1.00      5790
           3       0.28      0.10      0.15       401
           4       0.14      0.11      0.13       408
           5       0.20      0.29      0.24       375
           6       0.36      0.43      0.39       424
           7       0.28      0.27      0.28       401
           8       0.46      0.09      0.15       349
           9       0.36      0.13      0.19       334
          10       0.33      0.41      0.37       437
          11       0.29      0.26      0.28       393
          12       0.16      0.28      0.20       404
          13       0.57      0.52      0.54       280
          14       0.22      0.27      0.24       483
          15       0.17      0.39      0.23       410

    accuracy                           0.62     11720
   macro avg       0.33   

/home/azwad/anaconda3/envs/Final/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [22:03:23] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


              precision    recall  f1-score   support

           0       0.47      0.33      0.39       442
           1       0.41      0.35      0.38       389
           2       1.00      1.00      1.00      5790
           3       0.43      0.42      0.42       401
           4       0.47      0.42      0.45       408
           5       0.55      0.53      0.54       375
           6       0.55      0.75      0.64       424
           7       0.62      0.43      0.51       401
           8       0.46      0.37      0.41       349
           9       0.61      0.65      0.63       334
          10       0.71      0.75      0.73       437
          11       0.50      0.63      0.56       393
          12       0.45      0.50      0.48       404
          13       0.83      0.75      0.79       280
          14       0.50      0.51      0.51       483
          15       0.45      0.58      0.51       410

    accuracy                           0.76     11720
   macro avg       0.56   

,Model Name,Accuracy Score,F1 Score,Precision Score,Recall Score,Training Time (s),Inference Time (s)
0,Adaboost,0.753498,0.752882,0.754248,0.753498,50.8641,0.1136
1,Logistic Regression,0.621331,0.616804,0.634943,0.621331,7.7608,0.0015
2,Support Vector,0.626792,0.623525,0.659150,0.626792,31.9047,7.2170
3,Decision Tree,0.712201,0.710906,0.710688,0.712201,0.6190,0.0016
4,Random Forest,0.754437,0.753200,0.753577,0.754437,7.5146,0.1257
5,XGBoost,0.761263,0.758788,0.761376,0.761263,3.1141,0.0357
6,CatBoost,0.743003,0.739763,0.744476,0.743003,66.4980,0.1219
7,LightGBM,0.764078,0.761597,0.765066,0.764078,3.0143,0.1498


In [7]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score

# Binarize labels for multiclass
y_test_bin = label_binarize(y_test, classes=np.unique(y_test))  
y_prob = classifier.predict_proba(X_test)

auc_score = roc_auc_score(y_test_bin, y_prob, average='weighted', multi_class='ovr')
print("ROC AUC Score:", auc_score)


ROC AUC Score: 0.9780842722458016


**Ensemble**

In [8]:
import numpy as np
import pandas as pd
import time
from sklearn.ensemble import AdaBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Timing and performance DataFrame
metrics_sbtsk3 = pd.DataFrame(columns=[
    "Model Name", "Accuracy Score", "F1 Score", "Precision Score", "Recall Score",
    "Training Time (s)", "Inference Time (s)"
])

# Step 1: Train AdaBoost
start_train = time.time()
adaboost = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=20), n_estimators=50, random_state=42)
adaboost.fit(X_train, y_train)
train_time_ada = time.time() - start_train

# Inference (predict_proba)
start_infer = time.time()
adaboost_proba = adaboost.predict_proba(X_test)
infer_time_ada = time.time() - start_infer

adaboost_pred = np.argmax(adaboost_proba, axis=1)

# Confidence threshold
confidence_threshold = 0.8
max_probs = np.max(adaboost_proba, axis=1)
uncertain_indices = np.where(max_probs < confidence_threshold)[0]
certain_indices = np.where(max_probs >= confidence_threshold)[0]

# Step 2: Train LightGBM on uncertain predictions
start_train_lgbm = time.time()
lgbm = LGBMClassifier(random_state=42)
lgbm.fit(X_test[uncertain_indices], y_test.iloc[uncertain_indices])
train_time_lgbm = time.time() - start_train_lgbm

# Inference on uncertain predictions
start_infer_lgbm = time.time()
lgbm_preds = lgbm.predict(X_test[uncertain_indices])
infer_time_lgbm = time.time() - start_infer_lgbm

# Step 3: Combine predictions
final_predictions = adaboost_pred.copy()
final_predictions[uncertain_indices] = lgbm_preds

# Metrics
accuracy = accuracy_score(y_test, final_predictions)
f1 = f1_score(y_test, final_predictions, average='weighted')
precision = precision_score(y_test, final_predictions, average='weighted')
recall = recall_score(y_test, final_predictions, average='weighted')

# Total timing
total_train_time = train_time_ada + train_time_lgbm
total_infer_time = infer_time_ada + infer_time_lgbm

# Store results
new_metrics = pd.DataFrame([{
    "Model Name": "AdaBoost-LightGBM",
    "Accuracy Score": accuracy,
    "F1 Score": f1,
    "Precision Score": precision,
    "Recall Score": recall,
    "Training Time (s)": round(total_train_time, 4),
    "Inference Time (s)": round(total_infer_time, 4)
}])

metrics_sbtsk3 = pd.concat([metrics_sbtsk3, new_metrics], ignore_index=True)

# Show results
metrics_sbtsk3


/home/azwad/anaconda3/envs/Final/lib/python3.12/site-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000300 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4248
[LightGBM] [Info] Number of data points in the train set: 1519, number of used features: 47
[LightGBM] [Info] Start training from score -2.435458
[LightGBM] [Info] Start training from score -2.546684
[LightGBM] [Info] Start training from score -2.563634
[LightGBM] [Info] Start training from score -2.427968
[LightGBM] [Info] Start training from score -2.804019
[LightGBM] [Info] Start training from score -2.919088
[LightGBM] [Info] Start training from score -2.837171
[LightGBM] [Info] Start training from score -2.804019
[LightGBM] [Info] Start training from score -3.021742
[LightGBM] [Info] Start training from score -3.091701
[LightGBM] [Info] Start training from score -2.793208
[LightGBM] [Info] Start training from score -2.450610
[LightGBM] [Info] Start training from score -3.413784
[LightGBM]

/tmp/ipykernel_10969/3724069315.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_sbtsk3 = pd.concat([metrics_sbtsk3, new_metrics], ignore_index=True)


,Model Name,Accuracy Score,F1 Score,Precision Score,Recall Score,Training Time (s),Inference Time (s)
0,AdaBoost-LightGBM,0.848549,0.848209,0.849661,0.848549,53.2835,0.1502


In [9]:
import numpy as np
import pandas as pd
import time
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Initialize DataFrame for metrics including timing
metrics_sbtsk3 = pd.DataFrame(columns=[
    "Model Name", "Accuracy Score", "F1 Score", "Precision Score", "Recall Score",
    "Training Time (s)", "Inference Time (s)"
])

# Step 1: Train Logistic Regression
start_train_logreg = time.time()
logreg = LogisticRegression(penalty="l2", C=10, max_iter=10000, random_state=42)
logreg.fit(X_train, y_train)
train_time_logreg = time.time() - start_train_logreg

# Logistic Regression inference (predict_proba)
start_infer_logreg = time.time()
logreg_proba = logreg.predict_proba(X_test)
infer_time_logreg = time.time() - start_infer_logreg

logreg_pred = np.argmax(logreg_proba, axis=1)

# Confidence threshold
confidence_threshold = 0.8
max_probs = np.max(logreg_proba, axis=1)
uncertain_indices = np.where(max_probs < confidence_threshold)[0]
certain_indices = np.where(max_probs >= confidence_threshold)[0]

# Step 2: Train LightGBM on uncertain predictions
start_train_lgbm = time.time()
lgbm = LGBMClassifier(random_state=42)
lgbm.fit(X_test[uncertain_indices], y_test.iloc[uncertain_indices])
train_time_lgbm = time.time() - start_train_lgbm

# LightGBM inference on uncertain predictions
start_infer_lgbm = time.time()
lgbm_preds = lgbm.predict(X_test[uncertain_indices])
infer_time_lgbm = time.time() - start_infer_lgbm

# Step 3: Combine predictions
final_predictions = logreg_pred.copy()
final_predictions[uncertain_indices] = lgbm_preds

# Calculate metrics
accuracy = accuracy_score(y_test, final_predictions)
f1 = f1_score(y_test, final_predictions, average='weighted')
precision = precision_score(y_test, final_predictions, average='weighted')
recall = recall_score(y_test, final_predictions, average='weighted')

# Sum total training and inference times
total_train_time = train_time_logreg + train_time_lgbm
total_infer_time = infer_time_logreg + infer_time_lgbm

# Append results to metrics DataFrame
new_metrics = pd.DataFrame([{
    "Model Name": "Logistic Regression-LightGBM",
    "Accuracy Score": accuracy,
    "F1 Score": f1,
    "Precision Score": precision,
    "Recall Score": recall,
    "Training Time (s)": round(total_train_time, 4),
    "Inference Time (s)": round(total_infer_time, 4)
}])

metrics_sbtsk3 = pd.concat([metrics_sbtsk3, new_metrics], ignore_index=True)

# Display results
metrics_sbtsk3


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000390 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5432
[LightGBM] [Info] Number of data points in the train set: 5698, number of used features: 47
[LightGBM] [Info] Start training from score -2.570228
[LightGBM] [Info] Start training from score -2.689446
[LightGBM] [Info] Start training from score -7.549258
[LightGBM] [Info] Start training from score -2.676609
[LightGBM] [Info] Start training from score -2.651418
[LightGBM] [Info] Start training from score -2.723615
[LightGBM] [Info] Start training from score -2.598137
[LightGBM] [Info] Start training from score -2.663934
[LightGBM] [Info] Start training from score -2.810140
[LightGBM] [Info] Start training from score -2.864045
[LightGBM] [Info] Start training from score -2.739788
[LightGBM] [Info] Start training from score -2.674061
[LightGBM] [Info] Start training from score -2.648934
[LightGBM]

/tmp/ipykernel_10969/95593994.py:69: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_sbtsk3 = pd.concat([metrics_sbtsk3, new_metrics], ignore_index=True)


,Model Name,Accuracy Score,F1 Score,Precision Score,Recall Score,Training Time (s),Inference Time (s)
0,Logistic Regression-LightGBM,0.986348,0.986359,0.986585,0.986348,8.9681,0.108


In [10]:
import numpy as np
import pandas as pd
import time
from sklearn.svm import SVC
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Initialize metrics DataFrame with timing columns
metrics_sbtsk3 = pd.DataFrame(columns=[
    "Model Name", "Accuracy Score", "F1 Score", "Precision Score", "Recall Score",
    "Training Time (s)", "Inference Time (s)"
])

# Step 1: Train SVC
start_train_svc = time.time()
svc = SVC(kernel="linear", probability=True, random_state=42)
svc.fit(X_train, y_train)
train_time_svc = time.time() - start_train_svc

# SVC inference (predict_proba)
start_infer_svc = time.time()
svc_proba = svc.predict_proba(X_test)
infer_time_svc = time.time() - start_infer_svc

svc_pred = np.argmax(svc_proba, axis=1)

# Confidence threshold and low-confidence indices
confidence_threshold = 0.8
max_probs = np.max(svc_proba, axis=1)
uncertain_indices = np.where(max_probs < confidence_threshold)[0]
certain_indices = np.where(max_probs >= confidence_threshold)[0]

# Step 2: Train LightGBM on uncertain cases
start_train_lgbm = time.time()
lgbm = LGBMClassifier(random_state=42)
lgbm.fit(X_test[uncertain_indices], y_test.iloc[uncertain_indices])
train_time_lgbm = time.time() - start_train_lgbm

# LightGBM inference on uncertain cases
start_infer_lgbm = time.time()
lgbm_preds = lgbm.predict(X_test[uncertain_indices])
infer_time_lgbm = time.time() - start_infer_lgbm

# Step 3: Combine predictions
final_predictions = svc_pred.copy()
final_predictions[uncertain_indices] = lgbm_preds

# Calculate metrics
accuracy = accuracy_score(y_test, final_predictions)
f1 = f1_score(y_test, final_predictions, average='weighted')
precision = precision_score(y_test, final_predictions, average='weighted')
recall = recall_score(y_test, final_predictions, average='weighted')

# Sum training and inference times
total_train_time = train_time_svc + train_time_lgbm
total_infer_time = infer_time_svc + infer_time_lgbm

# Append results
new_metrics = pd.DataFrame([{
    "Model Name": "Support Vector-LightGBM",
    "Accuracy Score": accuracy,
    "F1 Score": f1,
    "Precision Score": precision,
    "Recall Score": recall,
    "Training Time (s)": round(total_train_time, 4),
    "Inference Time (s)": round(total_infer_time, 4)
}])

metrics_sbtsk3 = pd.concat([metrics_sbtsk3, new_metrics], ignore_index=True)

# Show results
metrics_sbtsk3


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000625 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5598
[LightGBM] [Info] Number of data points in the train set: 5896, number of used features: 47
[LightGBM] [Info] Start training from score -2.590720
[LightGBM] [Info] Start training from score -2.718450
[LightGBM] [Info] Start training from score -7.583417
[LightGBM] [Info] Start training from score -2.688068
[LightGBM] [Info] Start training from score -2.675676
[LightGBM] [Info] Start training from score -2.755103
[LightGBM] [Info] Start training from score -2.637024
[LightGBM] [Info] Start training from score -2.693068
[LightGBM] [Info] Start training from score -2.832705
[LightGBM] [Info] Start training from score -2.870888
[LightGBM] [Info] Start training from score -2.602096
[LightGBM] [Info] Start training from score -2.708220
[LightGBM] [Info] Start training from score -2.680615
[LightGBM]

/tmp/ipykernel_10969/1989598382.py:69: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_sbtsk3 = pd.concat([metrics_sbtsk3, new_metrics], ignore_index=True)


,Model Name,Accuracy Score,F1 Score,Precision Score,Recall Score,Training Time (s),Inference Time (s)
0,Support Vector-LightGBM,0.988225,0.988227,0.988401,0.988225,158.466,7.573


In [11]:
import time
from sklearn.tree import DecisionTreeClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import pandas as pd
import numpy as np

# Convert X_test and y_test to DataFrame/Series if needed
X_test_df = pd.DataFrame(X_test, columns=X.columns) if isinstance(X_test, np.ndarray) else X_test
y_test_df = pd.Series(y_test) if isinstance(y_test, np.ndarray) else y_test

# Step 1: Train Decision Tree
start_train_dt = time.time()
dt = DecisionTreeClassifier(random_state=42, min_samples_leaf=10, min_samples_split=5, max_depth=10, criterion='entropy')
dt.fit(X_train, y_train)
train_time_dt = time.time() - start_train_dt

# Predict with DT (inference)
start_infer_dt = time.time()
dt_proba = dt.predict_proba(X_test_df)
infer_time_dt = time.time() - start_infer_dt

dt_pred = np.argmax(dt_proba, axis=1)

# Confidence threshold and uncertain cases
confidence_threshold = 0.8
max_probs = np.max(dt_proba, axis=1)
uncertain_indices = np.where(max_probs < confidence_threshold)[0]
certain_indices = np.where(max_probs >= confidence_threshold)[0]

# Step 2: Train LightGBM on uncertain samples
start_train_lgbm = time.time()
lgbm = LGBMClassifier()
lgbm.fit(X_test_df.iloc[uncertain_indices], y_test_df.iloc[uncertain_indices])
train_time_lgbm = time.time() - start_train_lgbm

# LightGBM inference on uncertain samples
start_infer_lgbm = time.time()
lgbm_preds = lgbm.predict(X_test_df.iloc[uncertain_indices])
infer_time_lgbm = time.time() - start_infer_lgbm

# Combine predictions
final_predictions = dt_pred.copy()
final_predictions[uncertain_indices] = lgbm_preds

# Calculate metrics
accuracy = accuracy_score(y_test, final_predictions)
f1 = f1_score(y_test, final_predictions, average='weighted')
precision = precision_score(y_test, final_predictions, average='weighted')
recall = recall_score(y_test, final_predictions, average='weighted')

# Total training and inference time
total_train_time = train_time_dt + train_time_lgbm
total_infer_time = infer_time_dt + infer_time_lgbm

# Store metrics and timing in DataFrame
metrics_sbtsk3 = pd.DataFrame([{
    "Model Name": "Decision Tree-LightGBM",
    "Accuracy Score": accuracy,
    "F1 Score": f1,
    "Precision Score": precision,
    "Recall Score": recall,
    "Training Time (s)": round(total_train_time, 4),
    "Inference Time (s)": round(total_infer_time, 4)
}])

metrics_sbtsk3


/home/azwad/anaconda3/envs/Final/lib/python3.12/site-packages/sklearn/base.py:486: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000547 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5291
[LightGBM] [Info] Number of data points in the train set: 5481, number of used features: 47
[LightGBM] [Info] Start training from score -2.524543
[LightGBM] [Info] Start training from score -2.682117
[LightGBM] [Info] Start training from score -7.510431
[LightGBM] [Info] Start training from score -2.645464
[LightGBM] [Info] Start training from score -2.597776
[LightGBM] [Info] Start training from score -2.751110
[LightGBM] [Info] Start training from score -2.564038
[LightGBM] [Info] Start training from score -2.642896
[LightGBM] [Info] Start training from score -2.825218
[LightGBM] [Info] Start training from score -2.803908
[LightGBM] [Info] Start training from score -2.918683
[LightGBM] [Info] Start training from score -2.648038
[LightGBM] [Info] Start training from score -2.671507
[LightGBM]

,Model Name,Accuracy Score,F1 Score,Precision Score,Recall Score,Training Time (s),Inference Time (s)
0,Decision Tree-LightGBM,0.623038,0.622822,0.639612,0.623038,1.4214,0.0908


In [12]:
import numpy as np
import pandas as pd
import time
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Step 1: Train Random Forest (Timing)
start_train_rf = time.time()
rf = RandomForestClassifier()
rf.fit(X_train, y_train)
train_time_rf = time.time() - start_train_rf

# Inference with Random Forest
start_infer_rf = time.time()
rf_proba = rf.predict_proba(X_test)
rf_pred = np.argmax(rf_proba, axis=1)
infer_time_rf = time.time() - start_infer_rf

# Set confidence threshold
confidence_threshold = 0.8
max_probs = np.max(rf_proba, axis=1)
uncertain_indices = np.where(max_probs < confidence_threshold)[0]

# Step 2: Train LightGBM on uncertain cases (Timing)
start_train_lgbm = time.time()
lgbm = LGBMClassifier(random_state=42)
lgbm.fit(X_test[uncertain_indices], y_test.iloc[uncertain_indices])
train_time_lgbm = time.time() - start_train_lgbm

# Inference with LightGBM
start_infer_lgbm = time.time()
final_predictions = rf_pred.copy()
final_predictions[uncertain_indices] = lgbm.predict(X_test[uncertain_indices])
infer_time_lgbm = time.time() - start_infer_lgbm

# Step 3: Evaluate and store results
metrics_sbtsk3 = pd.DataFrame(columns=[
    "Model Name", "Accuracy Score", "F1 Score", "Precision Score", "Recall Score",
    "Training Time (s)", "Inference Time (s)"
])

accuracy = accuracy_score(y_test, final_predictions)
f1 = f1_score(y_test, final_predictions, average='weighted')
precision = precision_score(y_test, final_predictions, average='weighted')
recall = recall_score(y_test, final_predictions, average='weighted')

# Add result row
new_metrics = pd.DataFrame([{
    "Model Name": "Random Forest-LightGBM",
    "Accuracy Score": accuracy,
    "F1 Score": f1,
    "Precision Score": precision,
    "Recall Score": recall,
    "Training Time (s)": round(train_time_rf + train_time_lgbm, 4),
    "Inference Time (s)": round(infer_time_rf + infer_time_lgbm, 4)
}])

metrics_sbtsk3 = pd.concat([metrics_sbtsk3, new_metrics], ignore_index=True)

# Display results
metrics_sbtsk3


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000448 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5342
[LightGBM] [Info] Number of data points in the train set: 4995, number of used features: 47
[LightGBM] [Info] Start training from score -2.524728
[LightGBM] [Info] Start training from score -2.581298
[LightGBM] [Info] Start training from score -6.570283
[LightGBM] [Info] Start training from score -2.573393
[LightGBM] [Info] Start training from score -2.562949
[LightGBM] [Info] Start training from score -2.799165
[LightGBM] [Info] Start training from score -2.711058
[LightGBM] [Info] Start training from score -2.766800
[LightGBM] [Info] Start training from score -2.766800
[LightGBM] [Info] Start training from score -3.039729
[LightGBM] [Info] Start training from score -2.959365
[LightGBM] [Info] Start training from score -2.649725
[LightGBM] [Info] Start training from score -2.621790
[LightGBM]

/tmp/ipykernel_10969/1164823959.py:59: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_sbtsk3 = pd.concat([metrics_sbtsk3, new_metrics], ignore_index=True)


,Model Name,Accuracy Score,F1 Score,Precision Score,Recall Score,Training Time (s),Inference Time (s)
0,Random Forest-LightGBM,0.985239,0.985267,0.985483,0.985239,8.564,0.2262


In [13]:
import time
import numpy as np
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Convert X_test and y_test to DataFrame/Series if needed
X_test_df = pd.DataFrame(X_test, columns=X.columns) if isinstance(X_test, np.ndarray) else X_test
y_test_df = pd.Series(y_test) if isinstance(y_test, np.ndarray) else y_test

# Step 1: Train XGBoost
start_train_xgb = time.time()
xgb = XGBClassifier(eval_metric="mlogloss", objective="multi:softmax", num_class=len(y.unique()), random_state=42)
xgb.fit(X_train, y_train)
train_time_xgb = time.time() - start_train_xgb

# XGBoost inference
start_infer_xgb = time.time()
xgb_proba = xgb.predict_proba(X_test_df)
infer_time_xgb = time.time() - start_infer_xgb

xgb_pred = np.argmax(xgb_proba, axis=1)

# Threshold for low-confidence samples
confidence_threshold = 0.9
max_probs = np.max(xgb_proba, axis=1)
uncertain_indices = np.where(max_probs < confidence_threshold)[0]
certain_indices = np.where(max_probs >= confidence_threshold)[0]

# Step 2: Train LightGBM on uncertain samples
start_train_lgbm = time.time()
lgbm = LGBMClassifier(random_state=42)
lgbm.fit(X_test_df.iloc[uncertain_indices], y_test_df.iloc[uncertain_indices])
train_time_lgbm = time.time() - start_train_lgbm

# LightGBM inference on uncertain samples
start_infer_lgbm = time.time()
lgbm_preds = lgbm.predict(X_test_df.iloc[uncertain_indices])
infer_time_lgbm = time.time() - start_infer_lgbm

# Combine predictions
final_predictions = xgb_pred.copy()
final_predictions[uncertain_indices] = lgbm_preds

# Calculate metrics
accuracy = accuracy_score(y_test, final_predictions)
f1 = f1_score(y_test, final_predictions, average='weighted')
precision = precision_score(y_test, final_predictions, average='weighted')
recall = recall_score(y_test, final_predictions, average='weighted')

# Total training and inference time
total_train_time = train_time_xgb + train_time_lgbm
total_infer_time = infer_time_xgb + infer_time_lgbm

# Initialize or append to metrics DataFrame
metrics_sbtsk3 = pd.DataFrame(columns=["Model Name", "Accuracy Score", "F1 Score", "Precision Score", "Recall Score", "Training Time (s)", "Inference Time (s)"])

new_metrics = pd.DataFrame([{
    "Model Name": "XGBoost-LightGBM",
    "Accuracy Score": accuracy,
    "F1 Score": f1,
    "Precision Score": precision,
    "Recall Score": recall,
    "Training Time (s)": round(total_train_time, 4),
    "Inference Time (s)": round(total_infer_time, 4)
}])

metrics_sbtsk3 = pd.concat([metrics_sbtsk3, new_metrics], ignore_index=True)

metrics_sbtsk3


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000548 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5280
[LightGBM] [Info] Number of data points in the train set: 5245, number of used features: 47
[LightGBM] [Info] Start training from score -2.527160
[LightGBM] [Info] Start training from score -2.614388
[LightGBM] [Info] Start training from score -2.604025
[LightGBM] [Info] Start training from score -2.609193
[LightGBM] [Info] Start training from score -2.772017
[LightGBM] [Info] Start training from score -2.541583
[LightGBM] [Info] Start training from score -2.704244
[LightGBM] [Info] Start training from score -2.781205
[LightGBM] [Info] Start training from score -2.992876
[LightGBM] [Info] Start training from score -2.881451
[LightGBM] [Info] Start training from score -2.659669
[LightGBM] [Info] Start training from score -2.673386
[LightGBM] [Info] Start training from score -3.630557
[LightGBM]

/tmp/ipykernel_10969/1735620659.py:69: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_sbtsk3 = pd.concat([metrics_sbtsk3, new_metrics], ignore_index=True)


,Model Name,Accuracy Score,F1 Score,Precision Score,Recall Score,Training Time (s),Inference Time (s)
0,XGBoost-LightGBM,0.988225,0.988238,0.988465,0.988225,3.8073,0.1356


In [14]:
import time
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

# Convert X_test and y_test to DataFrame/Series if needed for safe indexing
X_test_df = pd.DataFrame(X_test, columns=X.columns) if isinstance(X_test, np.ndarray) else X_test
y_test_df = pd.Series(y_test) if isinstance(y_test, np.ndarray) else y_test

# Step 1: Train CatBoost
start_train_cat = time.time()
catboost = CatBoostClassifier(verbose=0, random_state=42)
catboost.fit(X_train, y_train)
train_time_cat = time.time() - start_train_cat

# CatBoost inference
start_infer_cat = time.time()
catboost_proba = catboost.predict_proba(X_test_df)
infer_time_cat = time.time() - start_infer_cat

catboost_pred = np.argmax(catboost_proba, axis=1)

# Set confidence threshold
confidence_threshold = 0.8
max_probs = np.max(catboost_proba, axis=1)
uncertain_indices = np.where(max_probs < confidence_threshold)[0]
certain_indices = np.where(max_probs >= confidence_threshold)[0]

# Step 2: Train LightGBM on uncertain cases
start_train_lgbm = time.time()
lgbm = LGBMClassifier(random_state=42)
lgbm.fit(X_test_df.iloc[uncertain_indices], y_test_df.iloc[uncertain_indices])
train_time_lgbm = time.time() - start_train_lgbm

# LightGBM inference on uncertain samples
start_infer_lgbm = time.time()
lgbm_preds = lgbm.predict(X_test_df.iloc[uncertain_indices])
infer_time_lgbm = time.time() - start_infer_lgbm

# Combine predictions
final_predictions = catboost_pred.copy()
final_predictions[uncertain_indices] = lgbm_preds

# Calculate metrics
accuracy = accuracy_score(y_test, final_predictions)
f1 = f1_score(y_test, final_predictions, average='weighted')
precision = precision_score(y_test, final_predictions, average='weighted')
recall = recall_score(y_test, final_predictions, average='weighted')

# Total training and inference times
total_train_time = train_time_cat + train_time_lgbm
total_infer_time = infer_time_cat + infer_time_lgbm

# Initialize or append to metrics DataFrame
metrics_sbtsk3 = pd.DataFrame(columns=["Model Name", "Accuracy Score", "F1 Score", "Precision Score", "Recall Score", "Training Time (s)", "Inference Time (s)"])

new_metrics = pd.DataFrame([{
    "Model Name": "CatBoost-LightGBM",
    "Accuracy Score": accuracy,
    "F1 Score": f1,
    "Precision Score": precision,
    "Recall Score": recall,
    "Training Time (s)": round(total_train_time, 4),
    "Inference Time (s)": round(total_infer_time, 4)
}])

metrics_sbtsk3 = pd.concat([metrics_sbtsk3, new_metrics], ignore_index=True)

metrics_sbtsk3


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000346 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5301
[LightGBM] [Info] Number of data points in the train set: 5369, number of used features: 47
[LightGBM] [Info] Start training from score -2.543392
[LightGBM] [Info] Start training from score -2.632560
[LightGBM] [Info] Start training from score -2.619689
[LightGBM] [Info] Start training from score -2.614587
[LightGBM] [Info] Start training from score -2.739072
[LightGBM] [Info] Start training from score -2.555311
[LightGBM] [Info] Start training from score -2.707864
[LightGBM] [Info] Start training from score -2.820076
[LightGBM] [Info] Start training from score -2.911643
[LightGBM] [Info] Start training from score -2.884614
[LightGBM] [Info] Start training from score -2.642976
[LightGBM] [Info] Start training from score -2.658808
[LightGBM] [Info] Start training from score -3.611663
[LightGBM]

/tmp/ipykernel_10969/1793199631.py:69: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  metrics_sbtsk3 = pd.concat([metrics_sbtsk3, new_metrics], ignore_index=True)


,Model Name,Accuracy Score,F1 Score,Precision Score,Recall Score,Training Time (s),Inference Time (s)
0,CatBoost-LightGBM,0.988652,0.988672,0.98889,0.988652,67.8026,0.112
